# Распознавание именованных сущностей (NER)

В этом примере мы попробуем обучить модель NER на датасете [Entity Annoteted Corpus](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) из Kaggle. Мы будем использовать файл [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) - при работе в Colab его необходимо загрузить.

In [1]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Подготовка набора данных 

Загрузим датасет:

In [2]:
df = pd.read_csv('../../../data/ner_dataset.zip',encoding='unicode-escape',compression='zip')
df.head()

NameError: name 'pd' is not defined

Посмотрим на список уникальных тегов и создадим словари подстановки, которые мы сможем использовать для преобразования тегов в номера классов:

In [8]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [9]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Теперь нам нужно сделать то же самое со словарём. Для простоты мы создадим словарный запас без учета частоты слов; в реальной жизни вы можете использовать векторизатор Keras и ограничить количество слов.

In [10]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Нам нужно создать набор данных предложений для обучения. Для начала добавим идентификаторы слов и тегов в исходный датасет:

In [1]:
df['word_id'] = df['Word'].apply(lambda x : word2id[x.lower()])
df['tag_id'] = df['Tag'].apply(lambda x : tag2id[x])

NameError: name 'df' is not defined

Теперь дополним колонку `Sentence #` с помощью метода `fillna(method='ffill')`, и затем сгруппируем датасет по номеру предложения, сконкатенировав id слов и токенов:

In [1]:
res = df.fillna(method='ffill').groupby('Sentence #').agg({ 'word_id' : list, 'tag_id' : list })
res

Для простоты мы дополним все предложения нулевыми токенами до максимальной длины. В реальной жизни мы, скорее всего, захотим использовать masking, и дополнять последовательности до максимальной длины только в рамках одного минибатча.

In [13]:
X_data = keras.preprocessing.sequence.pad_sequences(res['word_id'])
Y_data = keras.preprocessing.sequence.pad_sequences(res['tag_id'])
X_data.shape

## NER - это классификация токенов

Мы будем использовать двухслойную двунаправленную сеть LSTM для классификации токенов. Чтобы применить финальный полносвязный классификатор к каждому выходу последнего слоя LSTM, мы будем использовать конструкцию `TimeDistributed`, которая реплицирует один и тот же полносвязный слой на каждый из выходов LSTM на каждом шаге: 

In [14]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, mask_zero=True), # input_length=maxlen
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 104, 300)          9545400   
_________________________________________________________________
bidirectional (Bidirectional (None, 104, 200)          320800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 104, 200)          240800    
_________________________________________________________________
time_distributed (TimeDistri (None, 104, 17)           3417      
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
_________________________________________________________________


Обратите внимание, что мы явно указываем `maxlen` для нашего набора данных - если мы хотим, чтобы сеть эффективно обрабатывала последовательности переменной длины, нам нужно определять сеть более хитрым способом.

Давайте теперь обучим модель. Для демонстрации мы используем только одну эпоху обучения, но вы можете попробовать продолжить обучение в течение более длительного времени. Кроме того, может потребоваться выделить некоторую часть набора данных в качестве тестового датасета, чтобы измерить точность модели.

In [15]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 866s 573ms/step - loss: 0.0646 - acc: 0.9843


## Тестирование результата

Давайте теперь посмотрим, как работает наша модель распознавания сущностей на примере предложения: 

In [16]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [17]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> O
development -> O
institute -> O


## Выводы

Даже простая модель LSTM показывает разумные результаты в задачах NER. Однако, чтобы получить более хорошие результаты, вы можете использовать большие предварительно обученные языковые модели, такие как BERT. Обучение BERT для NER с использованием библиотеки Huggingface Transformers описано [здесь](https://huggingface.co/course/chapter7/2).